# Neural Network Training

Imports:

In [3]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor, KerasClassifier
from sklearn.model_selection import  GridSearchCV, KFold, train_test_split
from sklearn.preprocessing import MinMaxScaler
import numpy as np

Load datasets

In [4]:
trainDf = pd.read_csv("training_data_treated.csv")
testDf = pd.read_csv("test_data_treated.csv")

Scalling features

In [5]:
x_normal = trainDf[["delay_in_seconds","avg_temperature","avg_atm_pressure","avg_humidity","record_date_month","record_date_day","record_date_isWeekend","record_date_hour","N101","R206","N105","N206","N309","IC5","N310","EM579","N207-4"]]
y_normal = trainDf[["incidents"]]

scaler_x = MinMaxScaler(feature_range=(0,1)).fit(x_normal)
scaler_y = MinMaxScaler(feature_range=(0,1)).fit(y_normal)
x = pd.DataFrame(scaler_x.transform(x_normal[x_normal.columns]),columns=x_normal.columns)
y = y_normal

Building the model

In [ ]:
def buildModel(activation1='sigmoid',activation2='sigmoid',activation3='sigmoid',learning_rate=0.01, neurons = 32):
    model = Sequential()
    model.add(Dense(neurons,input_dim=16, activation=activation1))
    model.add(Dense(neurons*2, activation=activation2))
    model.add(Dense(neurons, activation=activation2))
    model.add(Dense(neurons/2, activation=activation3))
    model.add(Dense(5, activation='softmax'))

    model.compile(
        loss= tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        optimizer= tf.keras.optimizers.Adam(learning_rate),
        metrics= ['accuracy']
    )
    return model

Tuning the model

In [ ]:
model = buildModel()
TUNING = {
    'activation1': ['relu','sigmoid','tanh','softmax'],
    'activation2': ['relu','sigmoid','tanh','softmax'],
    'activation3': ['relu','sigmoid','tanh','softmax'],
    'learning_rate': [0.01,0.005,0.001],
    'neurons':[int(x) for x in np.linspace(start = 10, stop = 30, num = 10)]
}

kf = KFold(n_splits=5,shuffle=True,random_state=13122001)
model = KerasClassifier(build_fn=buildModel,epochs=20,batch_size=8)

grid_search = GridSearchCV(estimator=model, 
                           param_grid=TUNING,
                           cv=kf,
                           scoring = 'accuracy',
                           refit='True',
                           verbose=1)


#grid_search.fit(x,y,validation_split=0.2)


In [ ]:
from livelossplot import PlotLossesKerasTF

grid_search.best_estimator_.fit(x,y,epochs=120,validation_split=0.3,callbacks=[PlotLossesKerasTF()],verbose=1)

### Create a submission

In [ ]:
x_train = trainDf[["delay_in_seconds","avg_temperature","avg_atm_pressure","avg_humidity","record_date_month","record_date_day","record_date_isWeekend","record_date_hour","N101","R206","N105","N206","N309","IC5","N310","EM579","N207-4"]]
x_test = trainDf[["incidents"]]
y_test = testDf[["delay_in_seconds","avg_temperature","avg_atm_pressure","avg_humidity","record_date_month","record_date_day","record_date_isWeekend","record_date_hour","N101","R206","N105","N206","N309","IC5","N310","EM579","N207-4"]]
scaler_train = MinMaxScaler(feature_range=(0,17)).fit(x_train)
scaler_test=MinMaxScaler(feature_range=(0,17)).fit(y_test)



In [ ]:
grid_search.best_estimator_.fit(x_train,x_test,epochs=120,validation_split=0.3,callbacks=[PlotLossesKerasTF()],verbose=1)
y_pred = grid_search.best_estimator_.predict(y_test,batch_size=8)

In [ ]:
f = open("Submissions/ANNSub.csv", "w")

replace_map = {0:'None', 1:'Low', 2:'Medium',3:'High',4:'Very_High'}

print(y_pred.size)


f.write("RowId,Incidents\n")

for i in range(y_pred.size):
    f.write(str(i+1))
    f.write(",")
    f.write(replace_map[y_pred[i]])
    f.write("\n")